In [1]:
from IPython.display import display
from notebook_viewer import NotebookViewer
notebook = NotebookViewer()

In [2]:
from brainbuilder.utils import genbrain as gb
from brainbuilder.utils import bbp
import numpy as np

cells = gb.CellCollection()

# Brain Builder modules

## Region Select

In [3]:
from brainbuilder.select_region import select_hemisphere

In [4]:
annotation = gb.VoxelData.load_metaio("../data/P56_Mouse_annotation/annotation.mhd")
hierarchy = gb.load_hierarchy("../data/P56_Mouse_annotation/annotation_hierarchy.json")['msg'][0]
full_density = gb.VoxelData.load_metaio("../data/atlasVolume/atlasVolume.mhd")

In [5]:
region_name = gb.find_in_hierarchy(hierarchy, 'acronym', 'CA1')[0]['name']
print 'Building:', region_name

Building: Field CA1


In [6]:
region_layers_map = {
    'hippocampal fissure': 1,
    'Field CA1, stratum lacunosum-moleculare': 2,
    'Field CA1, stratum radiatum': 3,
    'Field CA1, pyramidal layer': 4,
    'Field CA1, stratum oriens': 5,
    'alveus': 6
}

region_layers_map = dict((gb.find_in_hierarchy(hierarchy, 'name', name)[0]['id'], (layerid,))
                         for name, layerid in region_layers_map.iteritems())

In [7]:
recipe_filename = "../data/hippo_recipe/builderRecipeAllPathways.xml"

density = bbp.load_recipe_density(recipe_filename, annotation, region_layers_map)

density.raw = select_hemisphere(density.raw)

In [8]:
notebook.show_volume('density', density)

## Positions

In [9]:
from brainbuilder.cell_positioning import cell_positioning

In [10]:
voxel_dimensions = full_density.voxel_dimensions
total_cell_count = 350000

In [11]:
cells.positions = cell_positioning(density, total_cell_count)

In [12]:
notebook.show_points('position', cells)

## Build.EI:  E-I ratios

In [13]:
from brainbuilder.assignment_synapse_class import assign_synapse_class_from_spatial_dist

### input parameters

In [14]:
recipe_filename = "../data/hippo_recipe/builderRecipeAllPathways.xml"
recipe_data = bbp.load_recipe(recipe_filename)

In [15]:
recipe_sdist = bbp.transform_recipe_into_spatial_distribution(annotation, recipe_data, region_layers_map)

### run module

In [16]:
sclass_sdist = recipe_sdist.reduce('synapse_class')

In [17]:
chosen_synapse_class = assign_synapse_class_from_spatial_dist(cells.positions, sclass_sdist)
cells.add_properties(chosen_synapse_class)

In [18]:
notebook.show_property('synapse_class', cells)

In [19]:
import numpy as np
vals, nums = np.unique(cells.properties.synapse_class, return_counts=True)
print '\n'.join('%s  total: %d  percentage: %.2f%%' % (n, t, p * 100)
                for n, t, p in zip(vals, nums, nums.astype(np.float) / total_cell_count))

excitatory  total: 318064  percentage: 90.88%
inhibitory  total: 31936  percentage: 9.12%


## Build.Composition.ME: METype for Soma

In [20]:
from brainbuilder.assignment_metype import assign_metype

#### mtypes

In [21]:
chosen_me = assign_metype(cells.positions, cells.properties.synapse_class, recipe_sdist)
cells.add_properties(chosen_me)

#### mtypes

In [22]:
notebook.show_property('mtype', cells)

#### etypes

In [23]:
notebook.show_property('etype', cells)

## Build.Placement: Morphology assignment

In [24]:
from brainbuilder.assignment_morphology import assign_morphology
from scipy.ndimage import distance_transform_edt

### input parameters

In [25]:
hippo_mask = gb.get_regions_mask_by_ids(annotation.raw, gb.collect_in_hierarchy(hierarchy, 'name', region_name, 'id'))

In [26]:
neurondb_filename = "../data/hippo_recipe/v4neuronDB.dat"
neurondb = bbp.load_neurondb_v4(neurondb_filename)

# "outside"  is tagged in the annotation_raw with 0
# This will calculate, for every voxel, the euclidean distance to
# the nearest voxel tagged as "outside" the brain
# TODO use something else for hippocampus
distance_to_pia = distance_transform_edt(hippo_mask)

In [27]:
bbp.reverse_region_layers_map(region_layers_map)

{(1,): [1063], (2,): [391], (3,): [415], (4,): [407], (5,): [399], (6,): [466]}

In [28]:
neuron_sdist = bbp.transform_neurondb_into_spatial_distribution(annotation,
                                                                 neurondb,
                                                                 region_layers_map,
                                                                 distance_to_pia,
                                                                 percentile=0.92)

### run module

In [29]:
chosen_morphology = assign_morphology(cells.positions, cells.properties[['mtype', 'etype']], neuron_sdist)
cells.add_properties(chosen_morphology)

### output

In [30]:
notebook.show_property('morphology', cells)

## Orientation assignment

### input

In [31]:
from brainbuilder.orientation_field_hippo import compute_orientation_field

orientation_field = compute_orientation_field(annotation, hierarchy, 'Field CA1')

In [32]:
from brainbuilder.utils import vector_fields as vf

sub_fields = vf.split_orientation_field(orientation_field)
notebook.show_vectors('X', select_hemisphere(sub_fields[0]), 5000, voxel_dimensions)
notebook.show_vectors('Y', select_hemisphere(sub_fields[1]), 5000, voxel_dimensions)
notebook.show_vectors('Z', select_hemisphere(sub_fields[2]), 5000, voxel_dimensions)

### run module

In [33]:
from brainbuilder.assignment_orientation import assign_orientations

cells.orientations = assign_orientations(cells.positions, orientation_field, voxel_dimensions)

### output

In [34]:
# notebook.show_placement('placement', cells)